In [2]:
import pony as orm
import os

In [3]:
database="media.sqlite"

In [4]:
if os.path.exists(database):
    os.unlink(database)

In [3]:
# Database of receipts.
filename = os.path.abspath(database)
db = orm.Database(
    provider='sqlite',
    filename=filename,
    create_db=True,
)
class Media(db.Entity):
    date = orm.Required(datetime.datetime)
    store = orm.Required(str)
    items = orm.Required(int)
    total = orm.Required(float)
    pdf   = orm.Required(str)
db.generate_mapping(create_tables=True)

In [4]:
# Cache receipt table in a csv.
csv = open("receipts.csv", "w")

In [5]:
# mPerks user & password on separate lines in .secrets file
with open(".secrets", "r") as fid:
    user, password = fid.read().splitlines()

In [6]:
# Create Firefox driver.
driver = webdriver.Firefox()
# Set wait time to 5s to allow network time.
driver.implicitly_wait(5)
# Go to the mPerks account login.
driver.get("https://accounts.meijer.com/manage/Account/mPerks#/form/user")
# Fill user & submit.
elem = driver.find_element_by_name("email")
elem.clear()
elem.send_keys(user)
elem.send_keys(Keys.RETURN)
# Fill password & submit.
elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(password)
elem.send_keys(Keys.RETURN)
# Wait for login to complete
driver.find_element_by_id("editStore")
# Go to the receipts page
driver.get("https://mperks.meijer.com/mperks/ShoppingTools#receipts")
driver.find_element_by_class_name("receipt-list")

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="129e9177-a3e8-4a3e-be16-29b8d53643bd", element="99d23a0c-12c7-4b2e-bdaf-dad93af75c80")>

In [7]:
# Continually click "View More" until all available receipts are shown.
while True:
    try:
        driver.find_element_by_class_name("viewMore").click()
    except ElementNotInteractableException:
        print("Done.")
        break
    except ElementClickInterceptedException:
        sleep(1)
    except:
        raise

Done.


In [8]:
receiptItems = driver.find_elements_by_class_name("receiptItem")

In [11]:
def processReceipt(receiptItem):
    date = receiptItem.find_element_by_class_name("c1").text
    store = receiptItem.find_element_by_class_name("c2").text
    items = receiptItem.find_element_by_class_name("c3").text
    total = receiptItem.find_element_by_class_name("c4").text
    receipt_pdf = receiptItem.find_element_by_class_name("c5").find_element_by_tag_name("a").get_attribute("href")
    
    
    date_regex_ = date_regex.findall(date)[0]
    date2 = "{} {} {}".format(
        date_regex_[0],
        date_regex_[1],
        date_regex_[2],
    )
    datetime_date = datetime.datetime.strptime(date2, "%b %d %Y")

    Receipt(
        date = datetime_date,
        store = store,
        items = int(items),
        total = float(total.split("$")[1]),
        pdf = receipt_pdf
    )
    csv.write("{},{},{},{},{}\n".format(date,
                                       store,
                                       items,
                                       total,
                                       receipt_pdf))
    db.commit()

In [12]:
for receiptItem in receiptItems:
    processReceipt(receiptItem)

In [13]:
csv.flush()

In [14]:
csv.close()

In [23]:
for receipt in orm.select(r for r in Receipt):
    download_pdf(receipt.pdf)